## This notebook will be focusing on deploying the model to Hugging Face spaces as a demo application.

## Deployed model link: https://huggingface.co/spaces/purplelord2003/Trash_Object_Detector


In [1]:
from pathlib import Path


# Create demos folder
trash_detector_demo_path = Path("demos/trash_detector/")

if trash_detector_demo_path.exists():
  print("Directory already exists")

else:
  trash_detector_demo_path.mkdir(parents=True,
                                 exist_ok=True)

In [2]:
examples_path = trash_detector_demo_path / "examples"
examples_path.mkdir(parents=True,
                    exist_ok=True)

## Create model.py script for demo app

In [3]:
%%writefile demos/trash_detector/model.py
import torch
from torchvision import models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def create_fasterrcnn_model():
  model_weights = models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
  model = models.detection.fasterrcnn_resnet50_fpn_v2(weights=model_weights)
  transforms = model_weights.transforms()
  num_classes = 61
  in_features = 1024
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
  return model, transforms

Writing demos/trash_detector/model.py


In [4]:
# Check versions of dependencies for requirements.txt
import torch
torch.__version__

'2.1.0+cu121'

In [5]:
import torchvision
torchvision.__version__

'0.16.0+cu121'

## Create requirements.txt

In [6]:
%%writefile demos/trash_detector/requirements.txt
torch==2.1.0
torchvision==0.16.0
gradio==3.41.0

Writing demos/trash_detector/requirements.txt


In [7]:
# Get best model saved state dict determined in previous notebook
!wget -P demos/trash_detector/ https://github.com/purplelord2003/Object-Detection-Model/raw/main/Demo%20preparation/fasterrcnn_resnet50_5_epochs_unaugmented.pth

--2024-02-06 10:22:02--  https://github.com/purplelord2003/Object-Detection-Model/raw/main/Demo%20preparation/fasterrcnn_resnet50_5_epochs_unaugmented.pth
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/purplelord2003/Object-Detection-Model/main/Demo%20preparation/fasterrcnn_resnet50_5_epochs_unaugmented.pth [following]
--2024-02-06 10:22:03--  https://media.githubusercontent.com/media/purplelord2003/Object-Detection-Model/main/Demo%20preparation/fasterrcnn_resnet50_5_epochs_unaugmented.pth
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174626786 (167M) [application/octet-stream

In [8]:
# Get class names txt file
!wget -P demos/trash_detector/ https://github.com/purplelord2003/Object-Detection-Model/raw/main/Demo%20preparation/class_names.txt

--2024-02-06 10:22:12--  https://github.com/purplelord2003/Object-Detection-Model/raw/main/Demo%20preparation/class_names.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/purplelord2003/Object-Detection-Model/main/Demo%20preparation/class_names.txt [following]
--2024-02-06 10:22:13--  https://raw.githubusercontent.com/purplelord2003/Object-Detection-Model/main/Demo%20preparation/class_names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 859 [text/plain]
Saving to: ‘demos/trash_detector/class_names.txt’

class_names.txt     100%[===================>]     859  --.-KB/s    in 0s      

2024

In [9]:
# Get all the class names
with open("demos/trash_detector/class_names.txt", "r") as f: # reading them in from class_names.txt
  class_names = [class_name.strip() for class_name in f.readlines()]

In [10]:
class_names

['Aluminium foil',
 'Battery',
 'Aluminium blister pack',
 'Carded blister pack',
 'Other plastic bottle',
 'Clear plastic bottle',
 'Glass bottle',
 'Plastic bottle cap',
 'Metal bottle cap',
 'Broken glass',
 'Food Can',
 'Aerosol',
 'Drink can',
 'Toilet tube',
 'Other carton',
 'Egg carton',
 'Drink carton',
 'Corrugated carton',
 'Meal carton',
 'Pizza box',
 'Paper cup',
 'Disposable plastic cup',
 'Foam cup',
 'Glass cup',
 'Other plastic cup',
 'Food waste',
 'Glass jar',
 'Plastic lid',
 'Metal lid',
 'Other plastic',
 'Magazine paper',
 'Tissues',
 'Wrapping paper',
 'Normal paper',
 'Paper bag',
 'Plastified paper bag',
 'Plastic film',
 'Six pack rings',
 'Garbage bag',
 'Other plastic wrapper',
 'Single-use carrier bag',
 'Polypropylene bag',
 'Crisp packet',
 'Spread tub',
 'Tupperware',
 'Disposable food container',
 'Foam food container',
 'Other plastic container',
 'Plastic gloves',
 'Plastic utensils',
 'Pop tab',
 'Rope & strings',
 'Scrap metal',
 'Shoe',
 'Squeeza

## Write app.py

In [11]:
%%writefile demos/trash_detector/app.py

import gradio as gr
import os
import torch
import torchvision
from model import create_fasterrcnn_model

# Set up class names
with open("class_names.txt", "r") as f: # reading them in from class_names.txt
  class_names = [class_name.strip() for class_name in f.readlines()]

# Create model and transforms
model, transforms = create_fasterrcnn_model()

# Load saved weights
model.load_state_dict(
    torch.load(f="fasterrcnn_resnet50_5_epochs_unaugmented.pth",
               map_location=torch.device("cpu")) # Hugging Face spaces free tier gives cpu
)

# Prediction function
def predict(img: gr.Image=None, iou_threshold: float=0.5, score_threshold: float=0.5):

  # Get dimensions of the image
  width, height = img.size[0], img.size[1]

  # Resize if too small
  if width < 500 or height < 500:
    scaling = 500 / min(width, height)
    img = img.resize((int(width*scaling), int(height*scaling)))
    width, height = img.size[0], img.size[1]
  average = (width + height) / 2 # Help us to determine size of labels later

  # Transform the target image
  img = transforms(img)

  # Evaluation mode and inference mode
  model.eval()
  with torch.inference_mode():
    predictions = model(img.unsqueeze(dim=0))
    pred = predictions[0]

  img = torch.tensor(img*255, dtype=torch.uint8)

  # Remove boxes based on IoU threshold (to prevent multiple detections of same object)
  remaining_idx = torchvision.ops.nms(boxes=pred['boxes'],
                      scores=pred['scores'],
                      iou_threshold=iou_threshold)

  boxes = []
  labels = []
  for idx in remaining_idx:
    if pred["scores"][idx] > score_threshold:
      boxes.append(pred["boxes"][idx])
      labels.append(f"{class_names[pred['labels'][idx]]}: {round(pred['scores'][idx].item() * 100, 1)}%")
  if boxes:
    boxes = torch.stack(boxes)
    img = torchvision.utils.draw_bounding_boxes(img,
                                  boxes,
                                  labels,
                                  width=3,
                                  font="16020_FUTURAM.ttf",
                                  font_size=int(0.025*average)
                                  )
  return img.permute(1,2,0).numpy() # Gradio accepts numpy nd array as output to convert to an image

# Gradio interface

title = "Trash Object Detector 🗑️"
description = "A Faster R-CNN model that detects trash objects from [60 classes](https://github.com/purplelord2003/Object-Detection-Model/raw/main/Demo%20preparation/class_names.txt). Try to adjust [IoU threshold](https://learnopencv.com/non-maximum-suppression-theory-and-implementation-in-pytorch/) to be lower if multiple detections of the same object detected. Try to adjust score threshold to be lower if desired object not detected. Examples are defaulted at 0.5 IoU threshold and 0.5 scores threshold, feel free to adjust them and observe the changes in detections!"
article = "Created at [Trash Object Detection Model notebook](https://github.com/purplelord2003/Object-Detection-Model/blob/main/Jupyter%20Notebooks/Trash_Object_Detection_Model.ipynb)."

# Create examples list
example_list = [["examples/" + example] for example in os.listdir("examples")]

inputs = [
    gr.Image(type="pil", label="Input Image"),
    gr.Slider(minimum=0, maximum=1, value=0.5, step=0.01, label="Intersection over Union (IoU) Threshold"),
    gr.Slider(minimum=0, maximum=1, value=0.5, step=0.01, label="Score Threshold")
]

outputs = [
    gr.Image(type="numpy", label="Output Image")
]

demo = gr.Interface(
    fn=predict,
    inputs=inputs,
    examples=example_list,
    outputs=outputs,
    title=title,
    description=description,
    article=article
)

# Launch the app
demo.launch()

Writing demos/trash_detector/app.py


In [12]:
# Get examples for demo
!wget https://github.com/purplelord2003/Object-Detection-Model/raw/main/Demo%20preparation/Example%20Pictures%20for%20demo/Pictures.zip

--2024-02-06 10:22:13--  https://github.com/purplelord2003/Object-Detection-Model/raw/main/Demo%20preparation/Example%20Pictures%20for%20demo/Pictures.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/purplelord2003/Object-Detection-Model/main/Demo%20preparation/Example%20Pictures%20for%20demo/Pictures.zip [following]
--2024-02-06 10:22:13--  https://media.githubusercontent.com/media/purplelord2003/Object-Detection-Model/main/Demo%20preparation/Example%20Pictures%20for%20demo/Pictures.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3028655 (2.9M) [application/zip]
Saving to

In [13]:
from zipfile import ZipFile

with ZipFile("Pictures.zip", "r") as zip_ref:
  zip_ref.extractall("demos/trash_detector/examples")

In [14]:
!rm -rf demos/trash_detector/examples/__MACOSX

In [15]:
!rm Pictures.zip

In [16]:
# Get a font for our labels to be written in
!wget -P demos/trash_detector/ https://ttfonts.net/sfonts/1/16020_FUTURAM.ttf

--2024-02-06 10:22:14--  https://ttfonts.net/sfonts/1/16020_FUTURAM.ttf
Resolving ttfonts.net (ttfonts.net)... 31.207.45.27
Connecting to ttfonts.net (ttfonts.net)|31.207.45.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38764 (38K) [application/octet-stream]
Saving to: ‘demos/trash_detector/16020_FUTURAM.ttf’

16020_FUTURAM.ttf   100%[===================>]  37.86K  --.-KB/s    in 0.04s   

2024-02-06 10:22:15 (952 KB/s) - ‘demos/trash_detector/16020_FUTURAM.ttf’ saved [38764/38764]



In [17]:
# Zip folder but exclude certain files
!cd demos/trash_detector && zip -r ../trash_detector.zip * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"

  adding: 16020_FUTURAM.ttf (deflated 34%)
  adding: app.py (deflated 56%)
  adding: class_names.txt (deflated 52%)
  adding: examples/ (stored 0%)
  adding: examples/3.jpg (deflated 0%)
  adding: examples/1.jpg (deflated 0%)
  adding: examples/2.jpg (deflated 0%)
  adding: fasterrcnn_resnet50_5_epochs_unaugmented.pth (deflated 7%)
  adding: model.py (deflated 50%)
  adding: requirements.txt (deflated 8%)


In [18]:
from google.colab import files
files.download("demos/trash_detector.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>